In [1]:
from nnunet_ext.network_architecture.nca.NCA2D import NCA2D
import torch
import torch.nn.functional as F

In [2]:
device = "cuda"

In [3]:
hidden_size0 = 3
nca0 = NCA2D(num_channels=2, num_input_channels=1, num_classes=0, hidden_size=hidden_size0, fire_rate=1, num_steps=10, use_norm=False)
nca0.to(device)

NCA2D(
  (fc0): Conv2d(4, 3, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Conv2d(3, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (conv): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=2, padding_mode=reflect)
  (batch_norm): Identity()
)

In [4]:
x = torch.randn(1,1,5,5, device=device)

In [5]:
state = nca0.make_state(x)
out = nca0.forward_internal(state)
out.sum().backward()

In [6]:
hidden_size2 = 1
nca2 = NCA2D(num_channels=1, num_input_channels=0, num_classes=0, hidden_size=hidden_size2, fire_rate=1, num_steps=10, use_norm=False)
nca2.fc0 = torch.nn.Conv2d(2 * (nca0.num_channels + nca2.num_channels), hidden_size2, kernel_size=1)
nca2.to(device)

NCA2D(
  (fc0): Conv2d(6, 1, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=same, padding_mode=reflect)
  (batch_norm): Identity()
)

In [7]:
with torch.no_grad():
    nca2.fc0.bias.zero_()
    nca0.fc0.bias.zero_()

In [8]:
nca0.requires_grad_(False);

In [9]:
state1 = torch.zeros(x.shape[0], nca0.num_channels - nca0.num_input_channels,
                    x.shape[2], x.shape[3], device=x.device)
state2 = torch.zeros(x.shape[0], nca2.num_channels, x.shape[2], x.shape[3], device=x.device)
state = [torch.cat([x, state1], dim=1), state2]
[s.shape for s in state]
start_state_sep = torch.cat([state[0], state[1]], dim=1)

delta_state0 = nca0.conv(state[0])
delta_state1 = nca2.conv(state[1])
delta_state1 = torch.cat([state[0], state[1], delta_state0, delta_state1], dim=1)
delta_state0 = torch.cat([state[0], delta_state0], dim=1)
conv_cat_sep = delta_state1
fc0_input_sep = delta_state0
delta_state0 = nca0.fc0(delta_state0)
delta_state1 = nca2.fc0(delta_state1)
fc0_0_out_sep = delta_state0
fc0_out_sep = torch.cat([delta_state0, delta_state1], dim=1)
delta_state0 = nca0.batch_norm(delta_state0)
delta_state1 = nca2.batch_norm(delta_state1)
delta_state0 = F.relu(delta_state0, inplace=False)
delta_state1 = F.relu(delta_state1, inplace=False)
relu_out_0_sep = delta_state0
relu_out_sep = torch.cat([delta_state0, delta_state1], dim=1)
assert (relu_out_sep[: , :3] == relu_out_0_sep).all()
delta_state0 = nca0.fc1(delta_state0)
delta_state1 = nca2.fc1(delta_state1)
temp_state0 = state[0][:, nca0.num_input_channels:] + delta_state0
temp_state0 = torch.cat([state[0][:, :nca0.num_input_channels], temp_state0], dim=1)
state[0] = temp_state0
state[1] = state[1] + delta_state1
#(state[0].sum() + state[1].sum()).backward()

In [10]:

assert (relu_out_sep[: , :3] == relu_out_0_sep).all()

In [11]:
merged_nca = NCA2D(num_channels=nca0.num_channels + nca2.num_channels, num_input_channels=nca0.num_input_channels, 
                   num_classes=16, hidden_size=hidden_size0 + hidden_size2, fire_rate=1, num_steps=10, use_norm=False)

In [12]:
merged_nca.conv.weight = torch.nn.Parameter(torch.cat([nca0.conv.weight, nca2.conv.weight], dim=0))
merged_nca.conv.bias = torch.nn.Parameter(torch.cat([nca0.conv.bias, nca2.conv.bias], dim=0))

In [13]:
merged_nca.fc0.weight.shape, nca0.fc0.weight.shape, nca2.fc0.weight.shape,

(torch.Size([4, 6, 1, 1]), torch.Size([3, 4, 1, 1]), torch.Size([1, 6, 1, 1]))

In [14]:
merged_nca.fc0.weight.shape, nca0.fc0.weight.shape, nca2.fc0.weight.shape,

(torch.Size([4, 6, 1, 1]), torch.Size([3, 4, 1, 1]), torch.Size([1, 6, 1, 1]))

In [15]:
nca0.fc0.weight[:,:,0,0], merged_nca.fc0.weight[:,:,0,0]

(tensor([[ 0.2447, -0.2836, -0.4057,  0.4301],
         [-0.1179, -0.1713, -0.1001,  0.4020],
         [-0.2696, -0.4988, -0.4154, -0.4492]], device='cuda:0'),
 tensor([[-0.2485, -0.0358,  0.2614,  0.1805,  0.3823,  0.1775],
         [-0.0643,  0.0398, -0.2004, -0.3039,  0.1240,  0.0107],
         [-0.1203,  0.1448,  0.1231,  0.0398, -0.2533,  0.0049],
         [ 0.0500,  0.3637, -0.3240,  0.2855,  0.1869,  0.4069]],
        grad_fn=<SelectBackward0>))

In [16]:
merged_nca.fc0.weight.shape, nca0.fc0.weight.shape, nca2.fc0.weight.shape

(torch.Size([4, 6, 1, 1]), torch.Size([3, 4, 1, 1]), torch.Size([1, 6, 1, 1]))

In [17]:
temp = torch.cat([torch.zeros(nca0.fc0.weight.shape[0], 2, 1, 1, device=nca0.fc0.weight.device), nca0.fc0.weight], dim=1)
merged_nca.fc0.weight = torch.nn.Parameter(torch.cat([temp, nca2.fc0.weight], dim=0))
merged_nca.fc0.bias = torch.nn.Parameter(torch.cat([nca0.fc0.bias, nca2.fc0.bias], dim=0))

In [18]:
temp0 = torch.cat([nca0.fc1.weight, torch.zeros((15, 12, 1, 1), device=device)], dim=1)
temp2 = torch.cat([torch.zeros((3, 32, 1, 1), device=device), nca2.fc1.weight], dim=1)
merged_nca.fc1.weight = torch.nn.Parameter(torch.cat([temp0, temp2], dim=0))
#merged_nca.fc1.bias = torch.nn.Parameter(torch.cat([nca0.fc1.bias, nca2.fc1.bias], dim=0))

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1 but got size 15 for tensor number 1 in the list.

In [ ]:
sanity_merged_out = torch.cat([state[0], state[1]], dim=1)[:,1:]

In [ ]:
merged_state = torch.zeros(x.shape[0], nca0.num_channels - nca0.num_input_channels + nca2.num_channels,
                    x.shape[2], x.shape[3], device=x.device)
merged_state = torch.cat([x, merged_state], dim=1)
start_state_merged = merged_state.clone()

delta_state = merged_nca.conv(merged_state)
# interleave to correct order!
delta_state = torch.cat([merged_state, delta_state], dim=1)
conv_cat_merged = delta_state
delta_state = merged_nca.fc0(delta_state)
delta_state = merged_nca.batch_norm(delta_state)
delta_state = F.relu(delta_state, inplace=False)
relu_out_merged = delta_state
#delta_state = merged_nca.fc1(delta_state)

In [ ]:
s = nca0.make_state(x)
start_state_0 = s.clone()
st = nca0.conv(s)
conv_out_0 = st.clone()
st = torch.cat([s, st], dim=1)
conv_cat_0 = st
st = nca0.fc0(st)
fc0_out_0 = st
st = nca0.batch_norm(st)
st = F.relu(st, inplace=False)
relu_out_0 = st
st = nca0.fc1(st)

In [ ]:
relu_out_0.shape, relu_out_sep.shape, relu_out_merged.shape, relu_out_0_sep.shape

(torch.Size([1, 3, 128, 128]),
 torch.Size([1, 4, 128, 128]),
 torch.Size([1, 4, 128, 128]),
 torch.Size([1, 3, 128, 128]))

In [ ]:
assert (start_state_merged == start_state_sep).all()
assert (start_state_merged[:, :2] == start_state_0).all()

assert (conv_cat_merged == conv_cat_sep).all()

assert (conv_cat_sep[:, :4] == conv_cat_0).all()
assert (conv_cat_0 == fc0_input_sep).all()

assert (fc0_0_out_sep == fc0_out_0).all()
assert (fc0_out_sep[:, :3] == fc0_0_out_sep).all()

assert (relu_out_sep[:, :3] == relu_out_0).all()
assert (relu_out_sep[:, :3] == relu_out_0_sep).all()

AssertionError: 

In [ ]:
conv_cat_merged == conv_cat_sep

tensor([[[[ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          ...,
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True]],

         [[ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          ...,
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True],
          [ True,  True,  True,  ...,  True,  True,  True]],

         [[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False,

In [ ]:
relu_out_0_sep

tensor([[[[0.0681, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.5039, 0.0000, 0.0000],
          [0.5467, 0.0000, 0.0000,  ..., 0.0916, 0.1515, 0.0000],
          ...,
          [1.0479, 0.0783, 0.0000,  ..., 0.5816, 0.0000, 0.9068],
          [0.0000, 0.0000, 0.0000,  ..., 0.2953, 0.0000, 0.0000],
          [0.8237, 0.0283, 0.0000,  ..., 0.4177, 0.0000, 0.0000]],

         [[0.0237, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0276,  ..., 0.2975, 0.0000, 0.0000],
          [0.3925, 0.0000, 0.0000,  ..., 0.0393, 0.0000, 0.0000],
          ...,
          [0.7442, 0.0000, 0.0000,  ..., 0.3209, 0.0000, 0.6467],
          [0.0000, 0.0000, 0.0000,  ..., 0.1772, 0.0000, 0.0000],
          [0.5806, 0.0000, 0.0000,  ..., 0.2841, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.1322,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.1019,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [ ]:
relu_out_merged

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.1939, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.3053, 0.0114, 0.0000],
          [0.0752, 0.0000, 0.0000,  ..., 0.0051, 0.5469, 0.0000],
          ...,
          [0.2951, 0.3084, 0.0000,  ..., 0.4586, 0.0000, 0.2275],
          [0.0000, 0.0000, 0.0000,  ..., 0.1140, 0.0000, 0.0966],
          [0.2207, 0.2972, 0.0000,  ..., 0.0869, 0.2472, 0.0000]],

         [[0.0256, 0.3246, 0.3820,  ..., 0.0000, 0.0813, 0.0439],
          [0.2061, 0.1401, 0.2462,  ..., 0.0000, 0.0081, 0.1667],
          [0.0000, 0.0831, 0.0380,  ..., 0.0135, 0.0000, 0.3977],
          ...,
          [0.0000, 0.0000, 0.1633,  ..., 0.0000, 0.3200, 0.0000],
          [0.1434, 0.0802, 0.5014,  ..., 0.0000, 0.1493, 0.0000],
          [0.0000, 0.0000, 0.5639,  ..., 0.0000, 0.0000, 0.1623]],

         [[0.1295, 0.4537, 0.5158,  ..., 0.0000, 0.1899, 0.1493],
          [0.3252, 0.2536, 0.3686,  ..., 0.0000, 0.1105, 0.2824],
          [0.0503, 0.1918, 0.1430,  ..., 0